In [48]:
# 📦 INSTALL LIBRARIES
!pip install -q gradio
!pip install -q git+https://github.com/huggingface/transformers.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [49]:
# 📥 IMPORTS
import gradio as gr
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer


In [50]:
# 🔧 LOAD MODEL AND TOKENIZER
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [51]:
# 🧠 DEFINE TEXT GENERATION FUNCTION
def generate_text(input):
    input_ids = tokenizer.encode(input, return_tensors="tf")
    beam_output = model.generate(
        input_ids,
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    output = tokenizer.decode(beam_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return ".".join(output.split(". ")[:-1]) + "."


In [52]:
# 🖥️ CREATE INTERFACE
output_text = gr.Textbox()
interface = gr.Interface(
    fn=generate_text,
    inputs="textbox",
    outputs=output_text,
    title="GPT-2 Text Generator",
    description="Give it a sentence and GPT-2 will complete it. Based on OpenAI's transformer model."
)

# 🚀 LAUNCH APP
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fc8678e244ba34a51c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [53]:
def advanced_generate_text(prompt, max_length, temperature, top_k):
    input_ids = tokenizer.encode(prompt, return_tensors="tf")
    output = model.generate(
        input_ids,
        max_length=int(max_length),
        temperature=float(temperature),
        top_k=int(top_k),
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded

# New Interface with sliders
interface = gr.Interface(
    fn=advanced_generate_text,
    inputs=[
        gr.Textbox(label="Prompt"),
        gr.Slider(50, 300, value=100, step=10, label="Max Length"),
        gr.Slider(0.1, 1.5, value=1.0, step=0.1, label="Temperature"),
        gr.Slider(0, 100, value=50, step=5, label="Top-k Sampling")
    ],
    outputs="text",
    title="🧠 Advanced GPT-2 Generator",
    description="Adjust the generation parameters for more control."
)

interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://59fcf51371e7af48d9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [2]:
# This saves your GPT-2 app code into app.py
app_code = '''
import gradio as gr
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
import tensorflow as tf

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

def generate_text(input):
    input_ids = tokenizer.encode(input, return_tensors="tf")
    beam_output = model.generate(
        input_ids,
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    output = tokenizer.decode(beam_output[0], skip_special_tokens=True)
    return ".".join(output.split(".")[:-1]) + "."

iface = gr.Interface(
    fn=generate_text,
    inputs="textbox",
    outputs="textbox",
    title="GPT-2 Text Generator",
    description="Enter some text and GPT-2 will generate continuation."
)

if __name__ == "__main__":
    iface.launch()
'''

with open("app.py", "w") as f:
    f.write(app_code)

# Save requirements.txt file
requirements = '''
gradio
tensorflow
transformers
'''

with open("requirements.txt", "w") as f:
    f.write(requirements)

# Save README.md file
readme = '''
# GPT-2 Text Generator App

This app uses GPT-2 and Gradio to generate text continuations.

Run with:

    pip install -r requirements.txt
    python app.py
'''

with open("README.md", "w") as f:
    f.write(readme)
